# Cookie monster

Consider Bob, he really likes cookies.  He wants to eat its way through piles of cookies.  He gets a maximum number of hours to eat all cookies.  Each hour, he eats from a single pile only.  Moreover, Bob wants to eat at his leasure, so he minimizes the number of cookies he eats per hour.  When time runs out, all cookies must be eaten.

For example, suppose Bob gets 4 piles of cookies with 2, 3, 7 and 4 cookies each.

* If he gets less than 4 hours, it is clear he can never eat all cookies, since he eats only from a single pile per hour.
* If he gets 4 hours though, he can eat 1 pile per hour, so he should eat at least 7 cookies per hour since that is the number of cookies in the largest pile.
* If he gets 5 hours, he can eat at a rate of 4 cookies per hour. He can spend 2 hours on the pile of 7 cookies, and 1 hour on each of the other 3 piles.
* However, if he is allowed to eat during 6 hours, he still has to eat 4 cookies per hour.

For 4 piles with 2, 3, 7, 4 cookies, the table below summarizes the rates as a function of the number of hours Bob gets to eat.

| hours to eat | eating rate  |
|--------------|--------------|
| 1            | not possible |
| 2            | not possible |
| 3            | not possible |
| 4            | 7            |
| 5            | 4            |
| 6            | 4            |
| 7            | 3            |
| 8            | 3            |
| 9            | 2            |
| 10           | 2            |
| 11           | 2            |
| 12           | 2            |
| 13           | 2            |
| 14           | 2            |
| 15           | 2            |
| 16           | 1            |
| 17           | 2            |
| ...          | ...          |

## Test data

It is useful to have the example above as a testcase.

In [1]:
piles = [2, 4, 7, 3]
expected_results = {
    1: None,
    2: None,
    3: None,
    4: 7,
    5: 4,
    6: 4,
    7: 3,
    8: 3,
    9: 2,
    10: 2,
    11: 2,
    12: 2,
    13: 2,
    14: 2,
    15: 2,
    16:1,
    17: 1,
}

## Requirements

In [2]:
import math
import random

## Implementation

It is useful to have a function that computes the required number of hours, given the piles and a eating rate.

In [3]:
def required_hours(piles, rate):
    return sum(map(lambda x: math.ceil(x/rate), piles))

The minimum eating rate is 1, the maximum rate is the size of the largest pile of cookies.  The simplest approach is to do a linear search over all the values starting from the maximal value to the minimal value, stopping when the number of required hours at a rate is less than or equal to the specified maximum number of hours.

In [4]:
def compute_min_eating_rate(piles, max_hours):
    if len(piles) > max_hours:
        raise ValueError(f'too many piles, at most {max_hours} can be eaten)')
    else:
        rate = max(piles)
        while rate > 0 and required_hours(piles, rate) <= max_hours:
            rate -= 1
        return rate + 1

In [5]:
for max_hours in range(4, 18):
    print(f'{max_hours:2d}: {compute_min_eating_rate(piles, max_hours)}')

 4: 7
 5: 4
 6: 4
 7: 3
 8: 3
 9: 2
10: 2
11: 2
12: 2
13: 2
14: 2
15: 2
16: 1
17: 1


It is useful to write a function to test the implementation, given the expected results.

In [6]:
def test_implementation(compute_min_eating_rate, expected_results, verbose=False):
    for max_hours, expected_rate in expected_results.items():
        if verbose:
            print(f'max_hours = {max_hours}')
        try:
            assert(compute_min_eating_rate(piles, max_hours) == expected_rate)
        except ValueError:
            assert(expected_rate is None)                                                           

In [7]:
test_implementation(compute_min_eating_rate, expected_results)

## Optimization

The efficiency can be improved by replacing the linear search by a bisection search.

In [8]:
def compute_min_eating_rate_bin_search(piles, max_hours):
    if len(piles) > max_hours:
        raise ValueError(f'too many piles, at most {max_hours} can be eaten)')
    else:
        min_rate, max_rate = 1, max(piles)
        while min_rate < max_rate:
            rate = (min_rate + max_rate)//2
            if required_hours(piles, rate) > max_hours:
                min_rate = rate + 1
            else:
                max_rate = rate
        return min_rate

In [9]:
test_implementation(compute_min_eating_rate_bin_search, expected_results)

## Benchmarking

To explore the difference in performance between the two implementations, it is useful to have a larger number piles and maximum eating time.

In [10]:
nr_piles = 500
max_pile_size = 1_000
max_hours = 2_000
piles = random.choices(range(1, max_pile_size), k=nr_piles)

In [11]:
compute_min_eating_rate(piles, max_hours)

142

In [12]:
%timeit compute_min_eating_rate(piles, max_hours)

21 ms ± 427 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit compute_min_eating_rate_bin_search(piles, max_hours)

242 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


It is clear that the bisection search is faster than linear search.